![alt text](../../figs/logoLEFT.png "CLDF") | CÂMARA LEGISLATIVA DO DISTRITO FEDERAL | ![alt text](../../figs/logoRIGHT.png "CLDF")
--- | :---: | ---
| **COORDENADORIA DE MODERNIZAÇÃO E INFORMÁTICA - CMI** | 
| **OFICINA ASI/LABHINOVA** | 
| **ANÁLISE EXPLORATÓRIA DE DADOS ABERTOS USANDO PYTHON** | 

# Plotando e Visualizando Gráficos Usando Bokeh

#### Cada jupyter-notebook possui "sua própria memória" assim objetos definidos em um não são acessíveis em outro. Ou seja, se você tentar acessar nosso "DataFrame" definido no notebook anterior, o notebook atual não vai reconhecer.

#### Então precisamos carregar tudo de novo para a memória. Portanto já adiantamos isso para você na próxima célula. Escrevemos os códigos necessários para Importar a biblioteca pandas, ler o arquivo csv e criar as colunas "divisao" e "data" criadas na etapa anterior da oficina:

In [10]:
import pandas as pd
df = pd.read_csv('bolsafamilia.csv')
df['divisao'] = df['valor'] / df['qtdBeneficiados']
df['data'] = pd.to_datetime(df['dataReferencia'])
df.head()

,id,dataReferencia,codIBGE,nomeIBGE,UFSigla,UFNome,valor,qtdBeneficiados,divisao,data
0,75908792,01/01/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11450358.0,63239,181.064818,2013-01-01
1,72208975,01/02/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11034212.0,61121,180.530620,2013-01-02
2,63909551,01/03/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11435012.0,61189,186.880191,2013-01-03
3,74709431,01/04/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,10762080.0,59505,180.860096,2013-01-04
4,73009570,01/05/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11761262.0,67777,173.528808,2013-01-05


#### Ótimo! Agora podemos continuar...

## Uma visualização efetiva de dados, pode ser a diferença entre sucesso e falha nas decisões de negócio. Por isso é importante que o Cientista de Dados aprenda a comunicar os dados por intermédio de gráficos.

![alt text](../../figs/bokeh-datavisualization.png "Visualização de Dados")

#### Esse conjunto de dados em particular apresenta os gastos do bolsa família em cada mês desde 2013 até 2020. Um exercício interessante seria, por exemplo, fazer um gráfico de barras mostrando o quanto foi gasto em cada ano. 

#### Mas antes do gráfico, precisamos agregar os valores por ano. Ou seja, somar para cada ano os valores dos gastos mensais.

#### Dessa forma, em primeiro lugar, precisamos criar a coluna que usaremos para realizar a agregação. Quereremos agregar por ano, então precisamos descobrir uma forma de criar uma coluna só com o ano de cada linha. Podemos fazer isso com o comando abaixo:

In [11]:
df['ANO'] = pd.DatetimeIndex(df['data']).year
df


,id,dataReferencia,codIBGE,nomeIBGE,UFSigla,UFNome,valor,qtdBeneficiados,divisao,data,ANO
0,75908792,01/01/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11450358.0,63239,181.064818,2013-01-01,2013
1,72208975,01/02/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11034212.0,61121,180.530620,2013-01-02,2013
2,63909551,01/03/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11435012.0,61189,186.880191,2013-01-03,2013
3,74709431,01/04/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,10762080.0,59505,180.860096,2013-01-04,2013
4,73009570,01/05/2013,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,11761262.0,67777,173.528808,2013-01-05,2013
...,...,...,...,...,...,...,...,...,...,...,...
80,71010608,01/09/2019,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,12459837.0,65270,190.896844,2019-01-09,2019
81,73009437,01/10/2019,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,12583058.0,65446,192.266265,2019-01-10,2019
82,69210574,01/11/2019,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,12110463.0,61517,196.863680,2019-01-11,2019
83,66307873,01/12/2019,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,12284381.0,62034,198.026582,2019-01-12,2019


#### O comando extrai o ano da coluna "data" e armazena numa nova coluna que nós chamamos de "ANO". Em seguida, vamos criar a agregação desejada usando essa nova coluna:

In [12]:
agregacao = df.groupby(['ANO']).sum()
agregacao

,id,codIBGE,valor,qtdBeneficiados,divisao
ANO,,,,,
2013,835704105,63601296,136882148.0,783516,2099.336388
2014,830411489,63601296,147621128.0,856821,2068.623134
2015,828606920,63601296,149440066.0,879250,2045.117841
2016,814407491,63601296,150499707.0,946384,1904.631152
2017,840109052,63601296,151730592.0,950536,1915.791988
2018,838704651,63601296,153313408.0,915218,2013.205879
2019,809106117,63601296,150125272.0,794778,2268.250440
2020,67007738,5300108,12109549.0,61776,196.023520


#### O comando de agregação é o "groupby". Precisamos informar o(s) campo(s) que serão usados para realizar a agregação. Ele devolve um novo DataFrame que nós armazenamos no objeto "agregacao".  Em seguida, informamos o tipo de operação que deve ser realizado, nesse caso o comando "sum()" indica que queremos somar os valores.

Aqui é importante fazer alguns comentários: Perceba que ele agregou por "ANO" somando os valores. Ele não quis nem saber, trouxe a soma do id, codIBGE, valor, qtdBeneficiados e divisao. 

Aqui o Cientista de Dados deve ter o bom senso de saber que somar "id" ou "codIBGE" que posuem mesmo "ANO" não faz sentido. Agora, somar o "valor" e a "qtdBeneficados" já faz mais sentido. Essa é a importância de entender o Domínio da Aplicação e entender o que os dados significam durante a análise exploratória.


#### Dessa forma, já temos um DataFrame chamado "agregacao" onde temos para cada ano, um total de valor gasto no Bolsa Familia. Vamos plotar nosso primeiro gráfico com esses dados. Queremos um gráfico de barras mostrando a progressão dos anos e o total do valor gasto.

#### Para plotar esse gráfico, escolhemos usar a biblioteca Bokeh apresentada anteriormente nos slides da oficina:

![alt text](../../figs/bokeh.png "Biblioteca Bokeh")

#### Antes de mais nada vamos pedir ao Bokeh para que os gráficos que iremos plotar sejam exibidos no próprio jupyter-notebook:

In [13]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

#### E agora vamos plotar o gráfico. Dessa vez não vou explicar o código passo-a-passo. Em vez disso, comentei o código para que seja possível entender o que é feito. Pode continuar: confia, executa e vamos ver o gráfico que é gerado:

In [47]:
# Importa as bibliotecas
from bokeh.models import ColumnDataSource, Grid
from bokeh.io import show
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models.formatters import NumeralTickFormatter
from bokeh.models import HoverTool

# Cria o source como um dicionário onde x é uma lista com os índices da agregação 
# e top é uma lista com os valores da agregação que criamos 
source = ColumnDataSource(dict(x=list(agregacao.index),top=list(agregacao['valor']),))

#Aqui definimos ferramentas que queremos usar na figura
TOOLS = "reset,crosshair,pan,wheel_zoom,box_zoom"
hover = HoverTool(
    tooltips=[
        ("Indice", "$index"),
        ("Ano", "@x"),
        ("Valor", "R$ @top{0,000,000.00}"),
    ],
    formatters={
        'Ano': 'printf',
        'Valor': 'printf',
    },
    mode='mouse'
)

# Criamos e configuramos a figura que vai receber o gráfico 
p = figure(tools=TOOLS, plot_width=700, plot_height=300, title="Gastos com Bolsa Família no DF por ano")
p.xaxis.axis_label = 'Ano'
p.yaxis.axis_label = 'Valor (R$)'
p.yaxis.formatter=NumeralTickFormatter(format="0,000,000.00")
p.background_fill_color = "lightcyan"
p.background_fill_alpha = 0.4
p.grid.grid_line_alpha = 0.9
p.add_tools(hover)

# Criamos um gráfico de barras verticais vbar dentro da figura definida anteriormente
p.vbar(x='x', top='top',bottom=0, width=0.5, fill_color="blue", source=source)

# Mostramos a figura
show(p)

#### Aí está! Seria interessante acessar o "Painel de Monitoriamento do Bolsa Família" (sim, esse portal existe) e comparar os valores que obtemos com aquelas informados nos painéis. 

O link é esse aqui: https://aplicacoes.mds.gov.br/sagirmps/bolsafamilia/index.html

Mas vamos deixar essa imagem aí abaixo obtida desse portal para você comparar os resultados que obtivemos com os do portal na data de criação dessa oficina:

![alt text](../../figs/bolsa-familia-jan2020.png "Resultados Bolsa Família Janeiro de 2020")

#### Veja só:

In [15]:
df.tail()

,id,dataReferencia,codIBGE,nomeIBGE,UFSigla,UFNome,valor,qtdBeneficiados,divisao,data,ANO
80,71010608,01/09/2019,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,12459837.0,65270,190.896844,2019-01-09,2019
81,73009437,01/10/2019,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,12583058.0,65446,192.266265,2019-01-10,2019
82,69210574,01/11/2019,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,12110463.0,61517,196.863680,2019-01-11,2019
83,66307873,01/12/2019,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,12284381.0,62034,198.026582,2019-01-12,2019
84,67007738,01/01/2020,5300108,BRASÍLIA,DF,DISTRITO FEDERAL,12109549.0,61776,196.023520,2020-01-01,2020


#### Veja bem:
- O Valor Repassado no Mês 01/2020 de "12.109.549,00" bate com a última linha do DataFrame;

- O Valor Anual Repassado Acumulado até 01/2020 de "150.125.272,00" bate com a penúltima barra do gráfico;

- O Valor Anual Repassado em 2018 de "153.313.408,00" bate com a ante-penúltima barra do gráfico;

- Agora, a Quantidade de Famílias Beneficiadas de 66.439 do Portal  não condiz com o que temos no DataFrame (última linha) de 61776. E assim, o valor da coluna "divisão" que corresponde ao Benefício Médio do Portal também não já que ele é obtido dividindo-se o Valor Repassado no Mês pelo total de famílias beneficiadas.

#### Vamos repetir o pedido JSON para ter certeza que não fomos nós que erramos?

In [18]:
from urllib.request import Request, urlopen
import json
siteApi = 'http://www.transparencia.gov.br/api-de-dados'

def getJsonData(strMesAno,strCodIBGE):
    method='bolsa-familia-por-municipio?mesAno={0}&codigoIbge={1}&pagina={2}'.format(strMesAno,strCodIBGE,1)
    req = Request(siteApi + '/' + method, headers={'User-Agent': 'Mozilla/5.0'})
    JSON = urlopen(req).read()
    return json.loads(JSON)

print('Função criada!')

Função criada!


In [19]:
dadosBolsaFamiliaDFJan2020 = getJsonData(202001,5300108)

In [20]:
dadosBolsaFamiliaDFJan2020

[{'id': 67007738,
  'dataReferencia': '01/01/2020',
  'municipio': {'codigoIBGE': '5300108',
   'nomeIBGE': 'BRASÍLIA',
   'nomeIBGEsemAcento': 'BRASILIA',
   'pais': 'BRASIL',
   'uf': {'sigla': 'DF', 'nome': 'DISTRITO FEDERAL'}},
  'tipo': {'id': 1,
   'descricao': 'Bolsa Família',
   'descricaoDetalhada': 'Bolsa Família'},
  'valor': 12109549.0,
  'quantidadeBeneficiados': 61776}]

#### Perceba que realmente, o dado que vem é:

In [21]:
dadosBolsaFamiliaDFJan2020[0]['quantidadeBeneficiados']

61776

#### Então, provavelmente, na data de criação dessa oficina, o Portal da Transparencia ainda não tinha atualizado seus dados! Se, durante a apresentação do curso os valores baterem, então que bom!

## Vamos plotar mais um gráfico?

#### Vamos aproveitar que a "agregacao" está prontinha e finalizar essa etapa da oficina plotando o Gráfico de Barras de Quantidade de Beneficiados por Ano:

In [48]:
# Importa as bibliotecas
from bokeh.models import ColumnDataSource, Grid
from bokeh.io import show
from bokeh.plotting import figure, show, output_file, output_notebook
from bokeh.models import HoverTool

# Cria o source como um dicionário onde x é uma lista com os índices da agregação 
# e top é uma lista com os dados da agregação que criamos 
source = ColumnDataSource(dict(x=list(agregacao.index),top=list(agregacao['qtdBeneficiados']),))

#Aqui definimos ferramentas de visualização que queremos usar na figura
TOOLS = "reset,crosshair,pan,wheel_zoom,box_zoom"
hover = HoverTool(
    tooltips=[
        ("Indice", "$index"),
        ("Ano", "@x"),
        ("Qtd", "@top{0,000,000}"),
    ],
    formatters={
        'Ano': 'printf',
        'Valor': 'printf',
    },
    mode='mouse'
)

# Criamos e configuramos a figura que vai receber o gráfico 
p = figure(tools=TOOLS, plot_width=700, plot_height=300, title="Total de Famílias Beneficiadas com Bolsa Família no DF por ano")
p.xaxis.axis_label = 'Ano'
p.yaxis.axis_label = 'Quantidade'
p.yaxis.formatter=NumeralTickFormatter(format="0,000,000.00")
p.background_fill_color = "lightyellow"
p.background_fill_alpha = 0.4
p.grid.grid_line_alpha = 0.9
p.add_tools(hover)


# Criamos um gráfico de barras verticais vbar dentro da figura definida anteriormente
p.vbar(x='x', top='top',bottom=0, width=0.5, fill_color="darkgreen", source=source)

# Mostramos a figura
show(p)

## Maravilha! Aprendemos a agregar e plotar gráficos! 

# Esperamos que essa oficina sirva de ponto de partida para você começar seus estudos em Ciência de Dados e seja capaz de obter dados e fazer você mesmo suas próprias análises exploratórias.